# 01_EDA_and_preprocessing
Parse Sheet1 (Rainfall Grid) and Sheet2 (Inflow) from `newflood.xlsx`.

In [ ]:
import pandas as pd
from pathlib import Path

DATA_XLSX = '../data/newflood.xlsx'
print('Update DATA_XLSX path if needed')

In [ ]:
xls = pd.read_excel(DATA_XLSX, sheet_name=None, header=None)
list(xls.keys())

In [ ]:
sheet1 = xls[list(xls.keys())[0]]
sheet1.head(6)

In [ ]:
lat_row = sheet1.iloc[0].values
lon_row = sheet1.iloc[1].values
print('Lat sample:', lat_row[:6])
print('Lon sample:', lon_row[:6])

In [ ]:
def parse_sheet1(df):
    import pandas as pd
    nrows, ncols = df.shape
    # find grid start by locating numeric lat/lon pairs
    grid_start = None
    for c in range(ncols):
        try:
            lat = float(df.iat[0,c])
            lon = float(df.iat[1,c])
            if not (pd.isna(lat) or pd.isna(lon)):
                grid_start = c
                break
        except Exception:
            continue
    if grid_start is None:
        grid_start = 3
    lat_vals = [float(x) if not pd.isna(x) else None for x in df.iloc[0, grid_start:].tolist()]
    lon_vals = [float(x) if not pd.isna(x) else None for x in df.iloc[1, grid_start:].tolist()]
    records = []
    for rid in range(2, nrows):
        row = df.iloc[rid]
        try:
            year = int(row[0]); month = int(row[1]); day = int(row[2])
            date = pd.Timestamp(year=year, month=month, day=day)
            data_row = row[grid_start:].tolist()
            for j, val in enumerate(data_row):
                lat = lat_vals[j] if j < len(lat_vals) else None
                lon = lon_vals[j] if j < len(lon_vals) else None
                try:
                    rainfall = float(val)
                except Exception:
                    rainfall = None
                records.append({'date':date, 'lat':lat, 'lon':lon, 'rainfall':rainfall})
        except Exception:
            continue
    return pd.DataFrame.from_records(records)

parsed = parse_sheet1(sheet1)
parsed.head()

In [ ]:
parsed['date'] = pd.to_datetime(parsed['date'])
parsed = parsed.dropna(subset=['lat','lon','rainfall']).reset_index(drop=True)
parsed.shape

In [ ]:
sheet2 = xls[list(xls.keys())[1]]
print('Sheet2 preview:')
sheet2.head()

In [ ]:
def parse_sheet2(df):
    import pandas as pd
    # try header row detection
    hdr = df.iloc[0].astype(str).str.lower().tolist()
    if any('date' in x for x in hdr) and any('inflow' in x for x in hdr):
        df.columns = df.iloc[0]
        df = df[1:]
    # find date and inflow columns
    cols = list(df.columns)
    low = [str(c).lower() for c in cols]
    date_col = None; inflow_col = None
    for c,lc in zip(cols, low):
        if 'date' in lc: date_col = c
        if 'inflow' in lc or 'cumec' in lc: inflow_col = c
    date_col = date_col or cols[0]
    inflow_col = inflow_col or (cols[1] if len(cols)>1 else cols[-1])
    df['date_parsed'] = pd.to_datetime(df[date_col], errors='coerce')
    df[inflow_col] = pd.to_numeric(df[inflow_col], errors='coerce')
    return df[['date_parsed', inflow_col]].rename(columns={'date_parsed':'date', inflow_col:'inflow'})

inflow = parse_sheet2(sheet2)
inflow.head()

In [ ]:
merged = pd.merge(parsed, inflow, on='date', how='left')
merged.head()

In [ ]:
merged.to_csv('processed_long_rainfall_v2.csv', index=False)
print('Saved processed_long_rainfall_v2.csv')